## Monte Carlo Results

In [83]:
setwd("/home/leoKraushaar/Documents/School/Year 3/Semester 2/STAT 413/Project/protests/")
library(dplyr)

In [84]:
data <- read.csv("data/montecarlo/2030_norm.csv")[, -1]
head(data)

,prov,season,retail,protests
,<chr>,<chr>,<dbl>,<int>
1,Alberta,Spring,10359631,9
2,British Columbia,Spring,9645388,25
3,Manitoba,Spring,4135184,7
4,New Brunswick,Spring,3615926,4
5,Newfoundland and Labrador,Spring,3065366,3
6,Northwest Territories,Spring,1920654,0


In [85]:
floor(nrow(data)/143)

[1] 2860

In [86]:
nrow(data)

[1] 408980

In [87]:
iterations <- rep(1:8392, each=143)

data$iter <- iterations[1:nrow(data)]

data[, c("season", "retail")] <- NULL

head(data)

,prov,protests,iter
,<chr>,<int>,<int>
1,Alberta,9,1
2,British Columbia,25,1
3,Manitoba,7,1
4,New Brunswick,4,1
5,Newfoundland and Labrador,3,1
6,Northwest Territories,0,1


In [88]:
interval <- function(X, alpha=0.05) {
    quantiles <- quantile(X, probs = c(alpha/2, 1/2, 1-(alpha/2)))
    return(quantiles)
}

In [89]:
interval_low <- function(X, alpha=0.05) {
    quantiles <- quantile(X, probs = c(alpha/2, 1/2, 1-(alpha/2)))
    return(as.numeric(quantiles[1]))
}

interval_high <- function(X, alpha=0.05) {
    quantiles <- quantile(X, probs = c(alpha/2, 1/2, 1-(alpha/2)))
    return(quantiles)
    return(as.numeric(quantiles[2]))
}

In [90]:
total_protests <- data %>% group_by(prov, iter) %>% summarise(total = sum(protests))

`summarise()` has grouped output by 'prov'. You can override using the
`.groups` argument.


In [91]:
intervals <- aggregate(total ~ prov, data=total_protests, FUN=interval)$total
provs <- aggregate(total ~ prov, data=total_protests, FUN=interval)$prov

In [92]:
results <- as.data.frame(cbind(prov=provs, intervals))
results

prov,2.5%,50%,97.5%
<chr>,<chr>,<chr>,<chr>
Alberta,103,116,131
British Columbia,193,218,245
Manitoba,63,71,80.5250000000001
New Brunswick,37,42,48
Newfoundland and Labrador,26,30,34
Northwest Territories,0,0,2
Nova Scotia,43,48,55
Nunavut,3,5,7
Ontario,668.475,755,849


In [93]:
init_data <- read.csv("data/merged_data.csv") |> subset(year==2023)
init_data <- init_data[, c("month", "GEO", "protests")]

init_tot <- aggregate(protests ~ GEO, data=init_data, sum)

results <- cbind("2023"=init_tot$protests, results)
results <- subset(results, select=c(2,1,3,4,5))


results[, "50%"] <- as.numeric(results[, "50%"])

results$pred.effect <- ifelse(results$`50%` > results$"2023", "inc.", "dec.")

results[, "97.5%"] <- as.numeric(results[, "97.5%"], type="integer") |> round()
results[, "2.5%"] <- as.numeric(results[, "2.5%"], type="integer") |> round()

results$sig.effect <- ifelse(results$`97.5%` > results$"2023" & results$`2.5%` < results$"2023", "N", "Y")

results

,prov,2023,2.5%,50%,97.5%,pred.effect,sig.effect
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,Alberta,139,103,116,131,dec.,Y
2,British Columbia,284,193,218,245,dec.,Y
3,Manitoba,118,63,71,81,dec.,Y
4,New Brunswick,61,37,42,48,dec.,Y
5,Newfoundland and Labrador,61,26,30,34,dec.,Y
6,Northwest Territories,6,0,0,2,dec.,Y
7,Nova Scotia,85,43,48,55,dec.,Y
8,Nunavut,11,3,5,7,dec.,Y
9,Ontario,627,668,755,849,inc.,Y


In [94]:
library(xtable)

In [95]:
print(xtable(results, caption="95% Prediction Intervals for Total Yearly Protests by Province, 2030", label="tab:total_protests", digits=0), type="latex", caption.placement="top", file="total_protests.tex")